##   Rainfall EDA (2021-2025)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose

import os
import gc
import warnings

In [ ]:
# Creating visuals folder
os.makedirs('visuals', exist_ok=True)

## Importing Data

In [ ]:
# Read data files

# Direct URL to the CSV file
url = "/kaggle/input/bhutan-rainfall-data-2021-2025/btn-rainfall-adm2-5ytd.csv"

# Read the CSV file into a pandas DataFrame
df = pd.read_csv(url, header=None, names=['date', 'adm2_id', 'ADM2_PCODE', 'n_pixels', 'rfh', 'rfh_avg', 'r1h', 'r1h_avg', 'r3h', 'r3h_avg', 'rfq', 'r1q', 'r3q', 'version'])
df.drop(axis=0,index=[0,1],inplace=True)


# Preview the data
display(df.head())


In [ ]:
url2="/kaggle/input/bhutan-rainfall-data-2021-2025/btn_adminboundaries_tabulardata.xlsx"
df2=pd.read_excel(url2)
# Select columns using a list
df2_sub=df2[['ADM2_EN','ADM2_PCODE','ADM1_EN','AREA_SQKM']]
df2_sub.head()

In [ ]:
merged_df = pd.merge(df, df2_sub, on='ADM2_PCODE', how='left')

merged_df.drop_duplicates(inplace=True)

## Data Exploration

In [ ]:
merged_df.head()

In [ ]:
print(merged_df.shape)
print(merged_df.columns)
print(merged_df.dtypes)
print(merged_df.isnull().sum())

In [ ]:
merged_df.info()

In [ ]:
merged_df['date'] = pd.to_datetime(merged_df['date'], errors='coerce')

## Transformation

In [ ]:
# Select columns with 'object' dtype, excluding 'ADM2_PCODE' and 'version'
object_cols = merged_df.select_dtypes(include='object').columns
cols_to_convert = [col for col in object_cols if col not in ['ADM2_PCODE', 'version','ADM2_EN','ADM1_EN']]


# Convert selected object columns to numeric, coercing errors
for col in cols_to_convert:
    merged_df[col] = pd.to_numeric(merged_df[col], errors='coerce')

# Display the data types after conversion
print(merged_df.info())

In [ ]:
# Create temporary columns for year and month
merged_df['year'] = merged_df['date'].dt.year
merged_df['month'] = merged_df['date'].dt.month

# Group by year and month and calculate the mean of rfh and rfh_avg
monthly_rainfall = merged_df.groupby(['year', 'month']).agg({
    'rfh': 'mean',

}).reset_index()

# Display the result
display(monthly_rainfall)

# # Drop the temporary columns
# df = df.drop(columns=['year', 'month'])

In [ ]:
merged_df[['rfh', 'rfh_avg', 'r1h',
       'r1h_avg', 'r3h', 'r3h_avg', 'rfq', 'r1q', 'r3q']].describe()

## Average rainfall

In [ ]:
# Create a combined year-month column for plotting
monthly_rainfall['year_month'] = pd.to_datetime(monthly_rainfall['year'].astype(str) + '-' + monthly_rainfall['month'].astype(str))
warnings.filterwarnings("ignore", category=FutureWarning)

plt.figure(figsize=(12, 5))
sns.lineplot(data=monthly_rainfall, x='year_month', y='rfh')
plt.title("Average Monthly Rainfall Over Time")
plt.xlabel("Date")
plt.ylabel("Average Rainfall (mm)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('/kaggle/working/visuals/Average_Monthly_rainfall_over_time.png')
plt.show()

In [ ]:
region_map = {
    'Thimphu': 'Western',
    'Paro': 'Western',
    'Punakha': 'Western',
    'Wangduephodrang': 'Western',
    'Haa': 'Western',
    'Gasa': 'Western',

    'Trashigang': 'Eastern',
    'Monggar': 'Eastern',
    'Lhuentse': 'Eastern',
    'Trashiyangtse': 'Eastern',
    'Pemagatshel': 'Eastern',

    'Samdrup Jongkhar': 'Southern',
    'Samtse': 'Southern',
    'Chukha': 'Southern',
    'Dagana': 'Southern',
    'Tsirang': 'Southern',
    'Sarpang': 'Southern',
    'Zhemgang': 'Southern',

    'Bumthang': 'Northern',
    'Trongsa': 'Central',
}
latitudes = {
    'Thimphu': 27.4661,
    'Paro': 27.433,
    'Punakha': 27.600,
    'Wangduephodrang': 27.400,
    'Haa': 27.500,
    'Gasa': 27.800,
    'Trashigang': 27.300,
    'Monggar': 27.300,
    'Lhuentse': 27.200,
    'Trashiyangtse': 27.300,
    'Pemagatshel': 26.900,
    'Samdrup Jongkhar': 26.700,
    'Samtse': 26.8990,
    'Chukha': 26.850,
    'Dagana': 27.100,
    'Tsirang': 27.100,
    'Sarpang': 26.900,
    'Zhemgang': 27.100,
    'Bumthang': 27.7500,
    'Trongsa': 27.500
}

merged_df['Latitude'] = merged_df['ADM1_EN'].map(latitudes)
#Climate also changes with latitude

merged_df['region'] = merged_df['ADM1_EN'].map(region_map)
print(merged_df['region'].value_counts())

#There are less data for Central and Northern belt

## Yearly rainfall plot (Region)




In [ ]:
#Group the merged dataframe by year and region and calculates the sum of rainfall (rfh column) for each region. 
yearly_rainfall_by_region = merged_df.groupby(['year', 'region'])['rfh'].sum().reset_index()

#Total rainfall in region from 2021-2025
total_rainfall_by_region = merged_df.groupby(['region'])['rfh'].sum().reset_index()


In [ ]:
display(total_rainfall_by_region)

In [ ]:
for index, row in yearly_rainfall_by_region.iterrows():
  year = row['year']
  region = row['region']
  rainfall = row['rfh']
  print(f"Year: {year}, Region: {region}, Total Rainfall: {rainfall}")

In [ ]:
plt.figure(figsize=(12, 6))
sns.barplot(data=yearly_rainfall_by_region, x='year', y='rfh', hue='region')
plt.title("Yearly Rainfall by Region")#shows yearly rainfall
plt.xlabel("Year")
plt.ylabel("Total Rainfall (rfh)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('/kaggle/working/visuals/yearly_rainfall_by_region.png')
plt.show()

The total yearly rainfall was calculated for each region by summing the rainfall (`rfh`) for each year and region combination.

* ##### Southern Belt :~386,000 mm(rfh yearly)
   Has subtropical climate and receives most rainfall .
* ##### Eastern Belt :  ~175,000 mm
    Moderate to high rainfall
* ##### western Belt: ~144,000 mm
     Rain shadow effect in some valleys
* ##### Central Belt :~11,000 mm
    Is more warmer and receives less rainfall
* ##### Northern Belt : ~5,000 mm
     Alpine; dry and cold
     Receives the Least Rainfall





## Rainfall by Season(Region)



In [ ]:
monthly_avg_rainfall_by_region = merged_df.groupby(['year', 'month', 'region'])['rfh'].mean().reset_index()

In [ ]:
plt.figure(figsize=(14, 7))
sns.lineplot(data=monthly_avg_rainfall_by_region, x='month', y='rfh', hue='region')
plt.title("Month-wise Average Rainfall by Region")
plt.xlabel("Month")
plt.ylabel("Average Rainfall (rfh)")
plt.xticks(range(1, 13), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.legend(title='Region')
plt.grid(True)
plt.tight_layout()
plt.savefig('/kaggle/working/visuals/monthly_avg_rainfall_by_region.png')
plt.show()

Bhutan has 4 main Season:
1. Spring (March – May)
2. Summer / Monsoon (June – August)
3. Autumn (September – November)
4. Winter (December – February)



| Region   | Spring                | Summer (Monsoon)   | Autumn        | Winter        |
|----------|-----------------------|--------------------|---------------|---------------|
| Northern | Cold start, warming   | Mild but wet       | Clear & crisp | Cold, snowy   |
| Central  | Pleasant              | Moderate rain      | Cool & dry    | Cold nights   |
| Western  | Cool & fresh          | Moderate rain      | Ideal         | Cold mornings |
| Eastern  | Early spring bloom    | Moderate rain      | Clear         | Cool          |
| Southern | Warm                  | 🌧️ Heavy rain    | Warm          | Mild          |

## Seasonality and Trend Analysis

In [ ]:
rainfall_1month_series=merged_df.groupby('date')['r1h'].mean().asfreq('MS').interpolate()
decomp = seasonal_decompose(rainfall_1month_series, model='additive', period=12)
decomp.plot()
plt.suptitle('Seasonal Decomposition of Monthly rainfall', fontsize=16)
plt.savefig('/kaggle/working/visuals/monthly_rainfall_decomposition.png')
plt.show()


Based on the plots :

#### Trend Plot:
 A  upward or downward slope would indicate a significant trend. In this case, the trend line is  relatively flat over the years.

#### Seasonality Plot:
 This plot highlights the repeating pattern in rainfall within each year. The plot should show a consistent pattern across the years, indicating the months with typically higher and lower rainfall. This is likely to show a peak during the monsoon season (June-August) and lower rainfall during the drier months.

#### Residual Plot:
This plot shows what remains of the time series after the trend and seasonal components have been removed. Ideally, the residuals should look like random noise, without any discernible patterns, trends, or seasonality. The plot_ts_resid function helps visualize if the residuals are centered around zero and if their variation is constant over time (homoscedasticity). If there are still patterns in the residuals, it suggests that the trend and seasonal components captured by the model are not fully explaining the variations in the rainfall data.



In [ ]:
def plot_ts_resid(x):
    x = x[x.notna()] # remove NAs
    fig, axes = plt.subplots(1, 2, figsize = (15, 5))
    fig = sns.lineplot(x=x.index,y=x, ax = axes[0])
    fig = sns.histplot(x, ax = axes[1], kde=True);
    plt.savefig('visuals/residue_plot.png')
    
    return None

In [ ]:
plot_ts_resid(decomp.resid)
